In [1]:
from fastapi import FastAPI
import joblib
import numpy as np
from pydantic import BaseModel

In [5]:
# Load trained model and scaler
model = joblib.load("credit_score_model.pkl")
scaler = joblib.load("scaler.pkl")

# Define FastAPI App
app = FastAPI()

# Input Schema
class CreditScoreInput(BaseModel):
    enq_L3m: float
    num_std: float
    time_since_recent_enq: float
    num_std_12mts: float
    enq_L6m: float
    AGE: int
    recent_level_of_deliq: float
    time_since_recent_deliquency: float
    Time_With_Curr_Empr: float
    time_since_recent_payment: float
    NETMONTHLYINCOME: float

@app.post("/predict")
def predict_credit_score(input_data: CreditScoreInput):
    # Convert input to array
    input_array = np.array([[input_data.enq_L3m, input_data.num_std, input_data.time_since_recent_enq,
                             input_data.num_std_12mts, input_data.enq_L6m, input_data.AGE,
                             input_data.recent_level_of_deliq, input_data.time_since_recent_deliquency,
                             input_data.Time_With_Curr_Empr, input_data.time_since_recent_payment,
                             input_data.NETMONTHLYINCOME]])
    
    # Scale input data
    scaled_input = scaler.transform(input_array)
    
    # Predict Credit Score
    prediction = model.predict(scaled_input)
    
    return {"predicted_credit_score": prediction[0]}
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://127.0.0.1:5500/frontend/index.html"],  # Change this to your frontend's domain in production
    allow_credentials=True,
    allow_methods=["POST"],
    allow_headers=["*"],
)
